# Project updates
In this notebook, we present the updates from the experiment results of our project.

## Data structure
The environments used in our project are simple `64x64` grid-worlds with different obstacles configurations.
Our objective is to achieve obstacle-free path planning from a `start` position to a `goal` position.
In total, we have **5000** different worlds (obstacle configurations) and for each configuration, we have **1000** different 
start and goal position pairs. So our dataset consists of **5,000,000** samples where 
the **1000** consecutive samples belong to a particular world.

<img src="../assets/images/environment_components.png" alt="Example Samples"/>  

As seen in the example samples above, our dataset consists of following data:
1. **Obstacle image:** `64x64` 2D image representing the obstacles of the world.
2. **Start image:** `64x64` 2D image representing the start position in the world.
3. **End image:** `64x64` 2D image representing the end position in the world.
4. **Path image:** `64x64` 2D image representing the path between the start and end positions in the world.
5. **Path vector**: `22x2` 2D vector representation of the `path image`. 

## Supervised Learning approaches

### Image-to-Image approach  
**Inputs:** 
- `64x64x2` 3D image with the given images stacked:  
  - `obstacle` image
  - `start` and `end` images concatenated

**Outputs:** 
- Path prediction as an `64x64x1` image

**Network architecture:**  
<img src="../assets/images/u_densenet_arch.png" alt="U-DenseNet-Architecture" width="1100"/>  

The hyperparameters for the above U-DenseNet architecture are:
- Number of dense blocks: 5
- Convolution blocks per dense blocK: 2
- Growth rate: 16
- Number of channels: 16

**Loss functions:**  

In our project, we experimented with the following loss functions:  
- **Cross entropy losses:**  

  **1. Cross Entropy (CE):**     
$$ L(Y, \hat{Y}) = -\frac{1}{N} \sum_{i=1}^{N} \sum_{w=1}^{W} \sum_{h=1}^{H} (Y_{iwh} log(\hat{Y}_{iwh}) - (1-Y_{iwh}) log(1-\hat{Y}_{iwh}))$$  
  where  
  $Y$ and $\hat{Y}$ are the set of the *true* and *predicted* path images respectively and the images in both set are normalized within the range $[0, 1]$.  
  
  **2. Weighted Cross Entropy (WCE):**   
  In the Weighted Cross Entropy loss, the positive examples are weighted by the coefficient $\beta \gt 0.0$.
$$ L(Y, \hat{Y}) = -\frac{1}{N} \sum_{i=1}^{N} \sum_{w=1}^{W} \sum_{h=1}^{H} (\underline{\beta} Y_{iwh} log(\hat{Y}_{iwh}) - (1-Y_{iwh}) log(1-\hat{Y}_{iwh}))$$
  
  **3. Balanced Cross Entropy (BCE):**   
  In the Balanced Cross Entropy (BCE) loss, both the positive and negative examples are weighted by the coefficient $\beta \gt 0.0$.
$$ L(Y, \hat{Y}) = -\frac{1}{N} \sum_{i=1}^{N} \sum_{w=1}^{W} \sum_{h=1}^{H} (\underline{\beta} Y_{iwh} log(\hat{Y}_{iwh}) - \underline{(1-\beta)} (1-Y_{iwh}) log(1-\hat{Y}_{iwh}))$$
  
  **4. Focal:**  
  In the Focal loss, the positive and negative examples are weighted even further by the coefficients $\beta \gt 0.0$ and $\gamma \geq 0.0$.
$$ L(Y, \hat{Y}) = -\frac{1}{N} \sum_{i=1}^{N} \sum_{w=1}^{W} \sum_{h=1}^{H} (\beta \underline{(1-\hat{Y}_{iwh})^{\gamma}} Y_{iwh} log(\hat{Y}_{iwh}) - (1-\beta) \underline{{\hat{Y}_{iwh}}^{\gamma}} (1-Y_{iwh}) log(1-\hat{Y}_{iwh}))$$  
  
- **Overlap losses:**  

  **1. Soft Dice (SD):**  
$$ L(Y, \hat{Y}) = 1 - \frac{1}{N} \sum_{i=1}^{N} \left(\frac{\sum_{w=1}^{W} \sum_{h=1}^{H} 2 Y_{iwh} \hat{Y}_{iwh}}{\sum_{w=1}^{W} \sum_{h=1}^{H} \left({Y_{iwh}}^2 + {\hat{Y}_{iwh}}^2\right)}\right)$$   
  
  **2. Tversky:**   
  The Tversky loss is the generalized version of the Dice loss as it introduces the weight coefficient $\beta \gt 0.0$.
$$ L(Y, \hat{Y}) = 1 - \frac{1}{N} \sum_{i=1}^{N} \left(\frac{\sum_{w=1}^{W} \sum_{h=1}^{H} 2 Y_{iwh} \hat{Y}_{iwh}}{\sum_{w=1}^{W} \sum_{h=1}^{H} \left(Y_{iwh} \hat{Y}_{iwh} + \beta (1-Y_{iwh}) \hat{Y}_{iwh} + (1-\beta) Y_{iwh} (1-\hat{Y}_{iwh})\right)}\right)$$  

**Results:**  
